### 1. 添加 rtc 模块

- 总线 1

```shell
2----5V
3----sda
5----scl
6----gnd
```

- 总线 0

```shell
2----5V
27----sda
28----scl
6----gnd
```

![](./images/raspberry-pin-assignments.jpeg)

### 2. 启用 i2c 接口

`raspi config` -> `Interfacing Options` -> `I2C` -> `enable`

### 3. 定义 i2c 设备

`/sys/class/i2c-adapter/i2c-1/new_device`

### 4. 读取时间、写入时间、同步时间

- 读取时间：`hwclock -r`

- 写入时间：`hwclock -w` 把系统时间写入 RTC 模块

- 同步时间：`hwclock -s` 把 RTC 模块时间写入系统时间

### 5. 参考

- [树莓派添加RTC时钟模块的方法](https://blog.csdn.net/huayucong/article/details/49982053)
- [Matrix - RTC/zh](http://wiki.friendlyarm.com/wiki/index.php/Matrix_-_RTC/zh)
- [linux hwclock命令参数及用法详解--linux显示/设置硬件时钟命令](http://www.linuxso.com/command/hwclock.html)
- [Adding a Real Time Clock to Raspberry Pi](https://cdn-learn.adafruit.com/downloads/pdf/adding-a-real-time-clock-to-raspberry-pi.pdf)

## 关于只读系统树莓派下使用 RTC 模块的测试

- [测试一：硬件时钟是否起作用（预设完毕，正常重启，检查）](#测试一：硬件时钟是否起作用)
- [测试二：硬件时钟是否行走准确（预设完毕，正常关机、断电30分钟后，重启检查）](#测试二：硬件时钟是否行走准确)
- [测试三：硬件时钟同步hwclock -s是否收到只读系统影响](#测试三：硬件时钟同步hwclock--s是否受到只读系统影响)


### <span style='color: #dd4b39;'>测试一：硬件时钟是否起作用</span>

- 已加入开机时间同步脚本为：

```shell
echo "Asynchronizing time ..."
   mount -o remount,rw /
   echo "ds1307 0x68" > /sys/class/i2c-adapter/i2c-1/new_device
   sleep 1  # wait for several time for /dev/rtc to be created
   hwclock -s
   mount -o remount,ro /
echo "Time asynchronized ."
```

1. 开机，只读系统
```shell
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
2. 系统时间设置为 20 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 20:00:00"
Tue  6 Nov 20:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 20:00:02 CST 2018
```
3. 系统转为可写
```shell
root@raspberrypi:~# mount -o remount,rw /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (rw,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
4. 硬件时间设置为 20 点
```shell
root@raspberrypi:~# hwclock -w
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:01:09 CST  -0.688271 seconds
```
5. 系统转为只读
```shell
root@raspberrypi:~# mount -o remount,ro /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
6. 系统时间设置为 21 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 21:00:00"
Tue  6 Nov 21:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 21:00:00 CST 2018
```
7. 检查时间：
    - 系统时间：21 点
    - 硬件时间：20 点
    - 系统类型：只读
    
```shell
root@raspberrypi:~# date
Tue  6 Nov 21:00:28 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:02:22 CST  -0.201743 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
8. 正常重启

```shell
root@raspberrypi:~# ps aux | grep iptalk | awk '{print $2}' | xargs kill -9
root@raspberrypi:~# reboot
root@raspberrypi:~# Connection to 192.168.1.199 closed by remote host.
Connection to 192.168.1.199 closed.
```
9. 检查时间：
    - 系统时间：\_\_20 点\_\_ √
    - 硬件时间：\_\_20 点\_\_ √
    - 系统类型：\_\_只 读\_\_ √

```shell
root@raspberrypi:~# date
Tue  6 Nov 20:04:48 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:04:53 CST  -0.305206 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)

```


### <span style='color: #dd4b39;'>测试二：硬件时钟是否行走准确</span>

- 已加入开机时间同步脚本为：

```shell
echo "Asynchronizing time ..."
   mount -o remount,rw /
   echo "ds1307 0x68" > /sys/class/i2c-adapter/i2c-1/new_device
   sleep 1  # wait for several time for /dev/rtc to be created
   hwclock -s
   mount -o remount,ro /
echo "Time asynchronized ."
```

1. 开机，只读系统
```shell
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
2. 系统时间设置为 20 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 20:00:00"
Tue  6 Nov 20:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 20:00:03 CST 2018
```
3. 系统转为可写
```shell
root@raspberrypi:~# mount -o remount,rw /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (rw,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
4. 硬件时间设置为 20 点
```shell
root@raspberrypi:~# hwclock -w
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:00:37 CST  -0.936800 seconds
```
5. 系统转为只读
```shell
root@raspberrypi:~# mount -o remount,ro /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
6. 系统时间设置为 21 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 21:00:00"
Tue  6 Nov 21:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 21:00:01 CST 2018
```
7. 检查时间：
    - 系统时间：21 点
    - 硬件时间：20 点
    - 系统类型：只读
    
```shell
root@raspberrypi:~# date
Tue  6 Nov 21:00:11 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:01:12 CST  -0.676217 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
8. 正常关机 30 分钟后重启

```shell
root@raspberrypi:~# ps aux | grep iptalk | awk '{print $2}' | xargs kill -9
root@raspberrypi:~# poweroff  # 然后断电
root@raspberrypi:~# Connection to 192.168.1.199 closed by remote host.
Connection to 192.168.1.199 closed.
```
9. 检查时间：
    - 系统时间：\_\_2?\_\_ 
    - 硬件时间：\_\_2?\_\_ 
    - 系统类型：\_\_只 读\_\_ √

```shell
root@raspberrypi:~# date
Tue  6 Nov 20:04:48 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:04:53 CST  -0.305206 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)

```



### <span style='color: #dd4b39;'>测试三：硬件时钟同步`hwclock -s`是否受到只读系统影响</span>

- 已加入开机时间同步脚本为：（去掉 mount -o ...）

```shell
echo "Asynchronizing time ..."
   #mount -o remount,rw /
   echo "ds1307 0x68" > /sys/class/i2c-adapter/i2c-1/new_device
   sleep 1  # wait for several time for /dev/rtc to be created
   hwclock -s
   #mount -o remount,ro /
echo "Time asynchronized ."
```

1. 开机，只读系统
```shell
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
2. 系统时间设置为 20 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 20:00:00"
Tue  6 Nov 20:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 20:00:03 CST 2018
```
3. 系统转为可写
```shell
root@raspberrypi:~# mount -o remount,rw /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (rw,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
4. 硬件时间设置为 20 点
```shell
root@raspberrypi:~# hwclock -w
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:00:48 CST  -0.933803 seconds
```
5. 系统转为只读
```shell
root@raspberrypi:~# mount -o remount,ro /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
6. 系统时间设置为 21 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 21:00:00"
Tue  6 Nov 21:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 21:00:00 CST 2018
```
7. 检查时间：
    - 系统时间：21 点
    - 硬件时间：20 点
    - 系统类型：只读
    
```shell
root@raspberrypi:~# date
Tue  6 Nov 21:00:08 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:01:37 CST  -0.025860 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
8. 正常重启
```shell
root@raspberrypi:~# ps aux | grep iptalk | awk '{print $2}' | xargs kill -9
root@raspberrypi:~# reboot
root@raspberrypi:~# Connection to 192.168.1.199 closed by remote host.
Connection to 192.168.1.199 closed.
```
9. 检查时间：
    - 系统时间：\_\_20 点\_\_ √
    - 硬件时间：\_\_20 点\_\_ √
    - 系统类型：\_\_只 读\_\_ √

```shell
root@raspberrypi:~# date
Tue  6 Nov 20:03:26 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:03:31 CST  -0.736403 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```


### <span style='color: #dd4b39;'>测试三：硬件时钟行走是否准确</span>

- 已加入开机时间同步脚本为：（去掉 mount -o ...）

```shell
echo "Asynchronizing time ..."
   #mount -o remount,rw /
   echo "ds1307 0x68" > /sys/class/i2c-adapter/i2c-1/new_device
   sleep 1  # wait for several time for /dev/rtc to be created
   hwclock -s
   #mount -o remount,ro /
echo "Time asynchronized ."
```

1. 开机，只读系统
```shell
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
2. 系统时间设置为 20 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 20:00:00"
Tue  6 Nov 20:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 20:00:02 CST 2018
```
3. 系统转为可写
```shell
root@raspberrypi:~# mount -o remount,rw /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (rw,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
4. 硬件时间设置为 20 点
```shell
root@raspberrypi:~# hwclock -w
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:00:43 CST  -0.506488 seconds
```
5. 系统转为只读
```shell
root@raspberrypi:~# mount -o remount,ro /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
6. 系统时间设置为 21 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 21:00:00"
Tue  6 Nov 21:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 21:00:01 CST 2018
```
7. 检查时间：
    - 系统时间：21 点
    - 硬件时间：20 点
    - 系统类型：只读
    
```shell
root@raspberrypi:~# date
Tue  6 Nov 21:00:15 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:01:45 CST  -0.577304 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
8. 正常关机 30 分钟后重启
```shell
root@raspberrypi:~# ps aux | grep iptalk | awk '{print $2}' | xargs kill -9
root@raspberrypi:~# poweroff
root@raspberrypi:~# Connection to 192.168.1.199 closed by remote host.
Connection to 192.168.1.199 closed.
```
9. 检查时间：
    - 系统时间：\_\_21:19\_\_×
    - 硬件时间：\_\_21:19\_\_ ×
    - 系统类型：\_\_只 读\_\_ √

```shell
root@raspberrypi:~# date
Tue  6 Nov 21:19:17 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 21:19:29 CST  -0.136734 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```


### <span style='color: #dd4b39;'>测试四：硬件时钟同步是否收到断电影响</span>

- 已加入开机时间同步脚本为：（去掉 mount -o ...）

```shell
echo "Asynchronizing time ..."
   #mount -o remount,rw /
   echo "ds1307 0x68" > /sys/class/i2c-adapter/i2c-1/new_device
   sleep 1  # wait for several time for /dev/rtc to be created
   hwclock -s
   #mount -o remount,ro /
echo "Time asynchronized ."
```

1. 开机，只读系统
```shell
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
2. 系统时间设置为 20 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 20:00:00"
Tue  6 Nov 20:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 20:00:02 CST 2018
```
3. 系统转为可写
```shell
root@raspberrypi:~# mount -o remount,rw /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (rw,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
4. 硬件时间设置为 20 点
```shell
root@raspberrypi:~# hwclock -w
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:00:43 CST  -0.506488 seconds
```
5. 系统转为只读
```shell
root@raspberrypi:~# mount -o remount,ro /
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
6. 系统时间设置为 21 点
```shell
root@raspberrypi:~# sudo date --s="2018-11-06 21:00:00"
Tue  6 Nov 21:00:00 CST 2018
root@raspberrypi:~# date
Tue  6 Nov 21:00:01 CST 2018
```
7. 检查时间：
    - 系统时间：21 点
    - 硬件时间：20 点
    - 系统类型：只读
    
```shell
root@raspberrypi:~# date
Tue  6 Nov 21:00:15 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:01:45 CST  -0.577304 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```
8. 断电 30 分钟后重启
```shell
*
```
9. 检查时间：
    - 系统时间：\_\_21:15\_\_×
    - 硬件时间：\_\_21:15\_\_ ×
    - 系统类型：\_\_只 读\_\_ √

```shell
root@raspberrypi:~# date
Tue  6 Nov 21:14:59 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 21:15:04 CST  -0.946571 seconds
root@raspberrypi:~# mount | grep ^/dev
/dev/mmcblk0p2 on / type ext4 (ro,noatime,data=ordered)
/dev/mmcblk0p1 on /boot type vfat (ro,relatime,fmask=0022,dmask=0022,codepage=437,iocharset=ascii,shortname=mixed,errors=remount-ro)
/dev/sda on /home/pi/hd type ext4 (rw,relatime,data=ordered)
```



### <span style='color: #dd4b39;'>测试前传一：瞎测</span>

系统时间设置为 21 点，硬件时钟设置为 20 点；

重启后，系统时间为 21 点，硬件时钟为 21 点。

```shell
root@raspberrypi:~# date
Tue  6 Nov 21:00:05 CST 2018
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 20:00:44 CST  -0.167680 seconds
root@raspberrypi:~# ps aux | grep iptalk | awk '{print $2}' | xargs kill -9
root@raspberrypi:~# reboot
root@raspberrypi:~# Connection to 192.168.1.199 closed by remote host.
Connection to 192.168.1.199 closed.
catcuts@catcuts-PC:project$ ssh root@192.168.1.199
root@192.168.1.199's password: 

The programs included with the Debian GNU/Linux system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Debian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent
permitted by applicable law.
root@raspberrypi:~# date
Tue  6 Nov 21:01:29 CST 2018
root@raspberrypi:~# ps aux | grep iptalk
root       831  0.2  0.3   5084  2880 ?        S    21:01   0:00 bash /home/pi/start_iptalk_on_rpi3.sh
root       941  7.6  3.1  65096 29520 ?        Sl   21:01   0:01 python /home/pi/hd/src/iptalk.py
root       974  0.0  0.1   4272  1844 pts/0    S+   21:01   0:00 grep iptalk
root@raspberrypi:~# hwclock -r
Tue 06 Nov 2018 21:01:50 CST  -0.978692 seconds
root@raspberrypi:~# echo '这说明：系统正常关机时，会把系统时间写入硬件时钟'
这说明：系统正常关机时，会把系统时间写入硬件时钟
root@raspberrypi:~# echo '如果系统时间一开始就是不想要的，那么关机后，硬件时钟就会被设置成不想要的时间'
如果系统时间一开始就是不想要的，那么关机后，硬件时钟就会被设置成不想要的时间
root@raspberrypi:~# echo '开机后，由于开机脚本执行了硬件时钟同步命令，所以硬件时钟时间会被写入系统时间'
开机后，由于开机脚本执行了硬件时钟同步命令，所以硬件时钟时间会被写入系统时间
root@raspberrypi:~# echo '所以系统最终的时间还是那个我们不想要的系统时间，而不是硬件时钟时间——它在系统关机时被重写了'
所以系统最终的时间还是那个我们不想要的系统时间，而不是硬件时钟时间——它在系统关机时被重写了
root@raspberrypi:~# echo '因此关键在于：系统关机时，是否真的把系统时间写入了硬件时钟，以及如何避开？'
因此关键在于：系统关机时，是否真的把系统时间写入了硬件时钟，以及如何避开？
```